In [14]:
sl = ['a', 'b', 'c', 'd']

In [15]:
%%timeit
list(map(lambda x: f'{x}_', sl))

297 ns ± 3.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [16]:
%%timeit
[f'{x}_' for x in sl]

204 ns ± 5.48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [17]:
a = ['x', 'y', 'y_', 'z_']
r = ['y_', 'z_']

In [20]:
tf = [tr in a for tr in r]
tf

[True, True]

In [21]:
tr = [tr for tr in r if tr in a]
tr

['y_', 'z_']

In [22]:
a.remove('x')

In [23]:
a

['y', 'y_', 'z_']

In [24]:
a.remove('z')

ValueError: list.remove(x): x not in list

In [25]:
'ThisString'.capitalize()

'Thisstring'

In [27]:
INPATH = '\home'
fr'{INPATH}\some\dir.py'

'\\home\\some\\dir.py'

In [28]:
rf'{INPATH}\some\dir.py'

'\\home\\some\\dir.py'

In [30]:
col_names = ['x', 'y', 'z']
my_cols = ['w', 'x', 'y', 'z', '0']

[col in my_cols for col in col_names]

[True, True, True]

In [35]:
def all_a_in_b(a, b):
    return set(a) - set(b) == set()

In [36]:
all_a_in_b(col_names, my_cols)

True

In [34]:
set(col_names) - set(my_cols)

set()

In [39]:
import pandas as pd

df = pd.DataFrame({'a': [1], 'b': 2})
df = df.rename(columns={'a': 'A', 'b': 'B', 'c': 'C'})
df

,A,B
0,1,2


In [41]:
df = pd.DataFrame({'a': pd.to_datetime('1976-11-03 01:23:45'), 'b': [0]})
df

,a,b
0,1976-11-03 01:23:45,0


In [43]:
df.a.dt.strftime('%Y-%m-%d')

0    1976-11-03
Name: a, dtype: object

In [44]:
df.a.dt.date

0    1976-11-03
Name: a, dtype: object

In [45]:
from collections import Counter

In [46]:
[item for item, count in Counter(['a', 'b', 'c', 'a', 'b', 'a']).items() if count > 1]

['a', 'b']

In [49]:
import numpy as np

np.array(col_names).any()

UFuncTypeError: ufunc 'logical_or' did not contain a loop with signature matching types (None, <class 'numpy.dtype[str_]'>) -> None

In [53]:
for x in ['abc', 'abcdefghij']:
    print('{:0<10}'.format(x)[8] + '{:0<10}'.format(x)[9])

00
ij


In [54]:
for x in ['abc', 'abcdefghij']:
    print(f'{x:0<10}'[8:10])

00
ij


In [58]:
from pathlib import Path, PureWindowsPath

INPATH = PureWindowsPath(r'L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping')
Path(INPATH) / 'trsqq_list.pkl'

PosixPath('L:\\/NaturalResources/Wetlands/Local Wetland Inventory/WAPO/EPA_2022_Tasks/Task 1 WD Mapping/trsqq_list.pkl')